## Adaptive HiTS

### Code written by Danish Rafiq and Asif Hamid

This script simulates the adaptive HiTS (AHiTS) method for several benchmark models. The code is a part of the paper: "Hierarchical deep learning based adaptive time-stepping of multiscale systems", Hamid A., Rafiq D., Nahvi SA., Bazaz MA. 2023, submitted to Nonlinear Dynamics

This script is build upon the multiscale AHiTs created by Yuying Liu (Liu Y, Kutz JN, Brunton SL.,2022 Hierarchical deep learning of multiscale differential equation time-steppers. Phil. Trans. R. Soc. A 380: 20210200.)

In [112]:
import os
import sys
import time
import torch
import numpy as np
import scipy.interpolate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import warnings
warnings.filterwarnings('ignore')

module_path = os.path.abspath(os.path.join('../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import ResNet as net
from aHiTs import *

In [113]:
# adjustables
dt = 0.01                     # time unit
noise = 0.0      #noise levels: 0.0, 0.01, 0.02, 0.05 ,0.1, 0.2
system = 'Hyperbolic'         # system name: 'Hyperbolic', 'Cubic', 'VanDerPol', 'Hopf', 'KS'

if system =='Hyperbolic':
    tol=1e-5
elif system =='Cubic':
    tol=5e-4
elif system =='VanDerPol':
    tol=8e-2
elif system =='Hopf':
    tol=5e-3
elif system =='Ks':
    tol=8e-6
else:
    print("please select a valid system")

# tolerance for AHiTs: 5e-3 for Hopf, 8e-2 for VanDerPol, 5e-4 for cubic, 1e-5 for hyperbolic, 8e-6 for KS (for more details, see discussion section of the paper)

invalid system


In [114]:
# path
data_dir = os.path.join('../../data/', system)
model_dir = os.path.join('../../models/', system)

# global const
ks = list(range(11))
step_sizes = [2**k for k in ks]

### load data & models

In [115]:
#load validation data set and test set
if system == 'KS':
    val_data = np.load(os.path.join(data_dir, 'data.npy')).T
    mean_values = val_data.mean(0)
    ranges = val_data.ptp(0)
    val_data = (val_data - mean_values)/ranges
    val_data = val_data[None, :, :]


    test_data = np.load(os.path.join(data_dir, 'data.npy')).T
    mean_values = test_data.mean(0)
    ranges = test_data.ptp(0)
    test_data = (test_data - mean_values) / ranges
    test_data = test_data[None, :, :]
else:
    val_data = np.load(os.path.join(data_dir, 'val_noise{}.npy'.format(noise)))
    test_data = np.load(os.path.join(data_dir, 'test_noise{}.npy'.format(noise)))

FileNotFoundError: [Errno 2] No such file or directory: '../../data/Hyperbolics\\val_noise0.0.npy'

In [ ]:
#load models
models = list()
for step_size in step_sizes:
    print('loading model_D{}.pt'.format(step_size))
    models.append(torch.load(os.path.join(model_dir, 'model_D{}_noise{}.pt'.format(step_size, noise)),map_location='cpu'))

# fix model consistencies trained on gpus (optional)
for model in models:
    model.device = 'cpu'
    model._modules['increment']._modules['activation'] = torch.nn.ReLU()

### benchmarks

In [ ]:
# shared info
n_steps = test_data.shape[1] - 1
t = [dt*(step+1) for step in range(n_steps)]
criterion = torch.nn.MSELoss(reduction='none')

In [ ]:
preds_mse = list()
times = list()
print('uniscale forecasting...')
for model in models:
    start = time.time()
    y_preds = model.uni_scale_forecast(torch.tensor(test_data[:, 0, :]).float(), n_steps=n_steps)
    end = time.time()
    times.append(end - start)
    preds_mse.append(criterion(torch.tensor(test_data[:, 1:, :]).float(), y_preds).mean(-1))
print('prediction recorded!')

In [ ]:
#AHiTs
#adaptive time-stepping calculation
start=time.time()
steps_used, index, indices_ahits = adaptive_multi_scale_forecast(val_data=val_data, n_steps=n_steps, models=models, best_mse=tol, dt=dt, step_sizes=step_sizes)
end=time.time()
ahits_offline = end - start
print('steps used: {}'.format(steps_used))
start=time.time()
# interative vectorized computations
y_preds_aHiTs, ahits_online = adaptive_multi_scale_online(val_data=val_data, test_data=test_data[:,0,:], n_steps=n_steps, models=models, dt=dt, step_sizes=step_sizes, steps_used=steps_used, index=index)
end=time.time()
ahits_time = end - start
print('AHiTs completed')

#calculate errors
mse_aHiTs = criterion(torch.tensor(test_data[:, 1:, :]).float(), y_preds_aHiTs[:,1:n_steps+1,:]).mean(-1)
aHiTs_err = mse_aHiTs.mean(0).detach().numpy()
norm_ahits=aHiTs_err.mean()

In [ ]:
# visualize forecasting error at each time step
norm_uni=list()
fig = plt.figure(figsize=(30, 10))
colors=iter(plt.cm.rainbow(np.linspace(0, 1, len(ks))))
mean_pointwise=list()

#Figure1: error plot
for k in range(len(preds_mse)):
    err = preds_mse[k]
    mean = err.mean(0).detach().numpy()
    norm_uni.append(mean.mean())  #mean
    mean_pointwise.append(mean)
    rgb = next(colors)
    plt.plot(t, np.log10(mean), linestyle='-', color=rgb, linewidth=5, label='$\Delta\ t$={}dt'.format(step_sizes[k]))
plt.plot(t, np.log10(aHiTs_err), linestyle='-', color='k', linewidth=6, label='AHiTs')
plt.legend(fontsize=30, loc='upper center', ncol=6, bbox_to_anchor=(0.5, 1.2))
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)

In [ ]:
#figure2: Time response
idx = 8 # (you need to change the code here accordingly as state variables are different for different systems)
t = np.linspace(0, (n_steps-1)*dt, n_steps)
if system == 'KS':
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.imshow(np.squeeze(test_data.T))
    ax1.set_title("Ground Truth")
    ax1.set_xlabel('t')
    ax1.set_ylabel('x')
    ax2.imshow(np.squeeze(y_preds_aHiTs.T))
    ax2.set_xlabel('t')
    ax2.set_ylabel('x')
    ax2.set_title("AHiTs")
else:
    fig = plt.figure(figsize=(30, 10))
    gs = gridspec.GridSpec(nrows=1, ncols=1, hspace=0.5)
    ax0 = fig.add_subplot(gs[0, :])
    if test_data.shape[2]==3:
        ax0.plot(t, test_data[idx, 1:, 0], 'r-', linewidth=10, label='x')
        ax0.plot(t, test_data[idx, 1:, 1], 'g-', linewidth=10, label='y')
        ax0.plot(t, test_data[idx, 1:, 2], 'b-', linewidth=10, label='z') #only in case of Hopf
        ax0.plot(t, y_preds_aHiTs[idx, :n_steps, 0].detach().numpy(), 'k--', linewidth=10, label='AHiTs')
        ax0.plot(t, y_preds_aHiTs[idx, :n_steps, 1].detach().numpy(), 'k--', linewidth=10)
        ax0.plot(t, y_preds_aHiTs[idx, :n_steps, 2].detach().numpy(), 'k--', linewidth=10)
    else:
        ax0.plot(t, test_data[idx, 1:, 0], 'r-', linewidth=10, label='x')
        ax0.plot(t, test_data[idx, 1:, 1], 'g-', linewidth=10, label='y')
        ax0.plot(t, y_preds_aHiTs[idx, :n_steps, 0].detach().numpy(), 'k--', linewidth=10, label='AHiTs')
        ax0.plot(t, y_preds_aHiTs[idx, :n_steps, 1].detach().numpy(), 'k--', linewidth=10)

    ax0.legend(fontsize=50, loc='upper right')
    ax0.tick_params(axis='both', which='major', labelsize=60)
    plt.show()

In [ ]:
#computational time
for i in range(len(times)):
    print('single scaled model (Dt={}): computing time {}s'.format(step_sizes[i]*dt, times[i]))
print('AHiTs:  {}s'.format(ahits_online))

#norms
norm_uni[0]=preds_mse[0][:,1:].mean()
for i in range(len(norm_uni)):
    print('MSE of NNTS (Dt={}): {}'.format(step_sizes[i]*dt, norm_uni[i]))
print('MSE of AHiTs: {}'.format(norm_ahits))